# Bibliotecas

In [ ]:
#!pip install beautifulsoup4
#!pip install selenium
#!pip install webdriver-manager
from urllib.request import urlopen
from urllib.error import HTTPError
from urllib.error import URLError
from bs4 import BeautifulSoup
from selenium import webdriver
import os
import sys
import numpy as np
import pandas as pd
from selenium.webdriver.common.keys import Keys
import time
import re
import xlwt
from datetime import datetime
import warnings
warnings.simplefilter("ignore")
import nltk
from pandas import DataFrame
from datetime import datetime, timedelta, date

# Função Recupera STATUS dos Projetos

In [ ]:
def BuscaOcor(ID, all ):
    # se all for TRUE não subescreve o TXT
    # se all for FALSE substreve o arquivo ou seja apaga todos as linhas do TXT
    
    global arqOcor
    
    try:    
        linha = 1
        htmlOcor = urlopen("http://qa.agricultura.gov.br/pmo/detail.php?id=" + ID)
        if all == True:
            arqOcor = open("ProjetosStatus.txt", "a")
        else:
            arqOcor = open("ProjetosStatus.txt", "w")
    except HTTPError as e:
        print(e)
    except URLError:
        print("Ligue a VPN")
    else:
        resOcor = BeautifulSoup(htmlOcor.read(),"html5lib")
        tagsOcor = resOcor.findAll("td")
        for tag in tagsOcor:
            if linha > 56:
                GravaLinha = tag.getText().replace('\n', "").replace('#', "")
                arqOcor.write(GravaLinha.strip() + "#")
                #arqOcor.write(tag.getText().strip() + "\n")
                #print(tag.getText().strip())
            linha += 1
        arqOcor.close()           

# Formatação e Gravação dos Status Para Cada Projeto  

In [ ]:
def FormataStatus():   
    cont = 0
    totcol = 19
    col = 0
    flag = 0
    Line4 = ""
    TxtProj = open("ProjetosStatus.txt", "r+")
    regProj = TxtProj.readline().split("#")
    regProj = np.array(regProj)

    while cont < len(regProj):
        line = regProj[cont].strip().replace('\n', "")
        if (line == "Descrição"):  
            cont += 1
            contCampo = 1
            indCampo = 1
            line = regProj[cont].strip().replace('\n', "")
            while line != "Descrição" and cont < len(regProj):
                line = regProj[cont].strip().replace('\n', "")
                try:
                    dataocor = datetime.strptime(line, '%d/%m/%Y').date()
                    if dataocor > date.today() - timedelta(days = 14):
                        cont += 1
                except ValueError:
                    if dataocor > date.today() - timedelta(days = 14):
                        if contCampo == 1:
                            line1 =  "[" + str(indCampo) + "] => " + str(f"{dataocor:%d/%m/%Y}")
                        if contCampo == 2:
                            line2 =  str(line)
                        if contCampo == 3:
                            line3 =  line1 + " - " + line2 + " - " + line
                            Line4 = Line4 + line3 + "\n"
                            contCampo = 0
                            indCampo += 1 
                        contCampo += 1
                cont += 1
        cont += 1
    TxtProj.close()
    return(Line4)
#    print(Line4)
#    print("**** FINALIZADO *****")

# Recuperando Todos os Projetos do SEI (Portfólio)

In [ ]:
try:
    html = urlopen("http://qa.agricultura.gov.br/pmo/portfolio.php")
    arq = open("ProjetosSEI.txt", "w")
    linha = 0
except HTTPError as e:
    print(e)
except URLError:
    print("Ligue a VPN")
else:
    res = BeautifulSoup(html.read(),"html5lib")
    tags = res.findAll("td")
    for tag in tags:
        if linha > 6:
            arq.write(tag.getText().strip() + "#")
            #print(tag.getText().strip())
        linha += 1
        #if linha > 200:
        #    break
arq.close()
print("**** FINALIZADO *****")


# Grava na Planilha EXCEL

In [ ]:
import win32com.client as win32

linha = 0
col = 0
ContCampo = 1

fname = "Projetos.xls"

try:
    ref_files = open("ProjetosSEI.txt", "r+")

    reg = ref_files.readline().split("#")
    regs = np.asarray(reg)

    if os.path.isfile(fname):
        os.remove(fname)
except FileNotFoundError:
    print("Arquivo TXT de Projetos Não Gerado !")
except OSError:
    print("Planilha Aberta ! Feche a planilha")
else:
    style0 = xlwt.easyxf('font: name Times New Roman, bold on, color-index red, height 210;', num_format_str ='#.##0,00')
    style1 = xlwt.easyxf('font: name Times New Roman, bold off,  height 180;', num_format_str ='#.##0,00')
    style2 = xlwt.easyxf('font: name Times New Roman, bold on, color-index blue, height 210;', num_format_str ='#.##0,00')

    wb = xlwt.Workbook()
    ws = wb.add_sheet('Projetos', cell_overwrite_ok=True)
    
    NrId = ""
    ws.col(19).width = 200 * 80
    ws.write(0, 19, "Status do Projeto", style0)
    totcol = 19
    for reg in regs:
        if totcol == col:
            if totcol == 18:
                print("Projeto nr.: " + str(NrId))
                BuscaOcor(NrId, False)
                StatusProj = FormataStatus()
                if StatusProj == "":
                    StatusProj = "Sem movimentação na Semana "
                    ws.col(19).width = 200 * 80
                    ws.write(linha, 19, StatusProj, style2)
                else:
                    ws.col(19).width = 200 * 80
                    ws.write(linha, 19, StatusProj, style1)
            linha += 1
            totcol = 18
            col = 0
        if totcol == 18:
            if col == 0:
                NrId = reg
            col_width = 256 * 30
            ws.col(col).width = col_width
            ws.write(linha, col, reg, style1)
        if totcol == 19:
            col_width = 256 * 20
            ws.col(col).width = col_width
            ws.write(linha, col, reg, style0)
        col += 1
        
        
        
        
    wb.save('Projetos.xls')
    ref_files.close()
    print("**** FINALIZADO *****")
